In [19]:
import numpy as np

In [20]:
def gini_index(groups, classes):
    n_instances = float(sum([len(group) for group in groups])) 
    gini = 0.0
    for group in groups:
        size = len(group)
        if size == 0:
            continue
        score = 0.0
        for class_val in classes:
            proportion = (group[:, -1] == class_val).sum() / size
            score += proportion ** 2
        gini += (1.0 - score) * (size / n_instances)
    return gini


In [21]:
def split_node(index, value, dataset):
    left = dataset[dataset[:, index] < value]
    right = dataset[dataset[:, index] >= value]
    return left, right

In [22]:
def get_best_split(dataset):
    class_values = np.unique(dataset[:, -1])
    best_index, best_value, best_score, best_groups = None, None, float('inf'), None
    for index in range(dataset.shape[1] - 1):
        for row in dataset:
            groups = split_node(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < best_score:
                best_index, best_value, best_score, best_groups = index, row[index], gini, groups
    return {'index': best_index, 'value': best_value, 'groups': best_groups}

In [23]:
def to_terminal(group):
    if len(group) == 0:  
        return None 
    outcomes = group[:, -1]
    return np.bincount(outcomes.astype(int)).argmax()

In [24]:
def build_tree(node, max_depth, min_size, depth):

    left, right = node['groups']
    del(node['groups'])  # Удаляем группы из узла, чтобы сохранить дерево компактным

    if depth >= max_depth:
        node['left'] = to_terminal(left)
        node['right'] = to_terminal(right)
        return

    if len(left) == 0:
        node['left'] = to_terminal(right)
        node['right'] = to_terminal(right)
        return
    if len(right) == 0:
        node['left'] = to_terminal(left)
        node['right'] = to_terminal(left)
        return

    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_best_split(left)
        build_tree(node['left'], max_depth, min_size, depth + 1)

    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_best_split(right)
        build_tree(node['right'], max_depth, min_size, depth + 1)

In [25]:
def decision_tree(train, max_depth, min_size):
    root = get_best_split(train)
    build_tree(root, max_depth, min_size, 1)
    return root

In [26]:
def predict(node, row):

    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

In [35]:
dataset = np.array([
    [2.771244718, 1.784783929, 0],
    [1.728571309, 1.169761413, 0],
    [3.678319846, 2.81281357, 0],
    [3.961043357, 2.61995032, 0],
    [2.999208922, 2.209014212, 0],
    [7.497545867, 3.162953546, 1],
    [9.00220326, 3.339047188, 1],
    [7.444542326, 0.476683375, 1],
    [10.12493903, 3.234550982, 1],
    [6.642287351, 3.319983761, 1]
])

In [36]:
tree = decision_tree(dataset, max_depth=1, min_size=7)

for row in dataset:
    prediction = predict(tree, row)
    print(f"Expected: {row[-1]}, Predicted: {prediction}")

Expected: 0.0, Predicted: 0
Expected: 0.0, Predicted: 0
Expected: 0.0, Predicted: 0
Expected: 0.0, Predicted: 0
Expected: 0.0, Predicted: 0
Expected: 1.0, Predicted: 1
Expected: 1.0, Predicted: 1
Expected: 1.0, Predicted: 1
Expected: 1.0, Predicted: 1
Expected: 1.0, Predicted: 1
